<img align="right" width="250" height="150" src="https://lh3.googleusercontent.com/p/AF1QipPWZQfa087JiVjutpUTVEGRnh6W214Wjm439gKQ=w1080-h608-p-no-v0">

# **Proyecto:** Retención de usuarios de Plataforma Digital

### Proceso de extracción, normalización y anonimización de datos

## **Fuentes de datos**

- `\\srvapwb12\SCG_INFORMACION_EXTERNA$\USUARIOS\GCIA BANCA DIGITAL` (Ana Belén Juncos)  
    - Usuarios de platadorma digital `ORA_Users.txt`
    - Crosseling de productos de Febrero a Abril `CLIPROD_CROSSELLING_YYYYMM.txt`
    - Transacciones monetarias y no monetares de Febrero a Abril: `Detalle_Trx_Monetarias_YYYYMM.txt` / `NoMonetarias_YYYYMM.txt`  
- `\\srvapwb12\SCG_INFORMACION_EXTERNA$\QV\USR\OUTPUT\Gestar` (Marcela Terreno)  
    - Transacciones mensuales de GESTAR - `Transacciones_Gestar_YYYYMM_NVO.txt`
- `\\srvapwb12\SCG_INFORMACION_EXTERNA$\QV\USR\OUTPUT\TURNERO` (Ana Laura Varas)  
    - Turnero - `748_YYYYMMDD_HHMM_Turnero_F.TXT`

## Declaraciones de librerías y declaraciones globales

In [1]:
# Importo las librerias necesarias
import pandas as pd
import datetime as dt
import numpy as np
import os

In [2]:
# Funciones globales de uso general

def dateparse_1(x):
    try:
        d = dt.datetime.strptime(x, '%d/%m/%Y')
    except:
        d = dt.datetime(1900, 1, 1)
    return d

def dateparse_2(x):
    try:
        d = dt.datetime.strptime(x, '%Y%m%d')
    except:
        d = dt.datetime(1900, 1, 1)
    return d

dateparse_dd_mm_yyyy = lambda x: dateparse_1(x)
dateparse_yyyymmdd = lambda x: dateparse_2(x)

In [3]:
# Variables Globales
DIR = os.path.abspath(r'..\Data')
PERIODOS = ["202202", "202203", "202204"]

## Depuración y generación del archivo de Usuarios

### Estructura del archivo

|Campo    |Descripción   |
|---    |---   |
|ID_USER    |Id de usuario (Hash de 32 bytes)   |
|EMAIL  |Correo electronico   |
|FIRST_NAME |Nombre   |
|LAST_NAME  |Apellido   |
|MOBILE_NUMBER  |Teléfono celular   |
|DOCUMENTO  |Documento (CUIT/CUIL)   |
|FechaCreacion  |Fecha de OnBoarding   |
|LAST_LOGIN |Último logín en la plataforma digital BANCON   |
|SegFactor  |Segundo factor de autenticación   |

### Proceso

In [4]:
# Lectura de archivo
file = DIR + r'\Usuarios\ORA_Users.txt'
df_us = pd.read_csv(file, 
    delimiter=';',
    encoding='latin-1',
    dtype={'DOCUMENTO': str, 'LAST_LOGIN': str, 'SegFactor': str},
    parse_dates=['FechaCreacion', 'LAST_LOGIN'],
    date_parser=dateparse_dd_mm_yyyy,
    usecols=['ID_USER', 'DOCUMENTO', 'FechaCreacion', 'LAST_LOGIN', 'SegFactor'])
df_us.shape

(917541, 5)

In [5]:
# Elimino los registros con fecha de último logín 1900-01-01
df_us = df_us[df_us['LAST_LOGIN'] != dt.datetime(1900, 1, 1)]

In [6]:
# Averiguar si hay o no CUIT/CUILs duplicados
ids = df_us['DOCUMENTO']
df_us[ids.isin(ids[ids.duplicated()])].sort_values('DOCUMENTO')

,ID_USER,DOCUMENTO,FechaCreacion,LAST_LOGIN,SegFactor
484554,ec0258dd773d4e518c261ca4a1291a0e,20043106261,2020-12-20,2021-09-10,SMS
584347,a40b6b59c911450b892700b8a7549b0a,20043106261,2021-11-09,2022-06-03,Softoken
128921,ca859587dc2f4368a0a91e2e583d34be,20046189958,2019-09-09,2019-09-12,SMS
114931,a8e526c2f12d40dcaf6b733df6f0fbf3,20046189958,2019-08-21,2022-05-31,SMS
479361,79a200d4ad454a63a25c61cc492ab9b7,20049293209,2021-03-02,2022-06-03,Softoken
...,...,...,...,...,...
898028,bbfdf324ce194746968c10eb15795361,27960094676,2020-02-13,2021-03-28,SMS
235572,a5c980c815654b3fadb0e58ec0458840,27960159832,2020-02-28,2021-03-18,SMS
814383,a841dbca361a4a4f9e7622c3171ecc46,27960159832,2021-04-06,2022-06-02,Softoken
537490,c9d18053509a4af1bcd52e6df537edc7,30710898967,2021-08-11,2021-08-12,No tiene segundo factor


In [7]:
# Hay duplicados. Me quedo con el registro con la fecha de último login mas reciente
df_us = df_us.sort_values('LAST_LOGIN').groupby('DOCUMENTO').tail(1)
df_us.shape

(903291, 5)

In [8]:
# Grabo
file = DIR + r'\Export\Usuarios.csv'
df_us[['ID_USER', 'FechaCreacion', 'LAST_LOGIN', 'SegFactor']].to_csv(file, index=False)

In [9]:
# Borro lo que ya no es necesario para econnomizar memoria
df_us.drop(['FechaCreacion', 'LAST_LOGIN', 'SegFactor'], axis=1, inplace=True)

## Depuración y generación del archivo de Crosseling

### Estructura del archivo

|Campo    |Descripción   |  
|---    |---    |
|PERIODO    |Período   |  
|CUITCLIE   |CUIT del cliente    | 
|RELACION   |Relación (TIT=Titular de cuenta/ADI=Adicional)| 
|TIPOIMPN   |Tipo de impuesto - **Ver de Excluir en todo caso tomar los consumidores finales**   |  
|NATUJURI   |Naturaleza Jurídica (Código=27) - Excluir   |  
|SUCURSAL   |Sucursal   |  
|BCRASECT   |Clasificación de Sector según BCRA (**PRIV**)  | 
|BCRASEGO   |Sector de Gobierno (Código=0)  |  
|GRUPO_CLIENTE  |Grupo de Cliente   | 
|GRUPOGENERAL   |Solo tomar **"Individuo sin actividad comercial"**   | 
|SUBGRUPO   |Subgrupo - **Incluir**   | 
|SEGMENTO_SUELDOS   |Agrupar los JUBILADOS y SUELDOS ADM*   | 
|SEGMENTO_BANCAS    |**No va - Excluir variable**  | 
|FUM    |Fecha de último movimiento   | 
|ES_CLIENTE |**Excluir columna - Siempre es 1** (1=Si/0=No)   |  
|ES_CAPITA  |**Si cobra sueldo en el banco - Convenio de haberes - EPEC** (1=Si/0=No)   |  
|CL_ACTIVO_FUM  |Identifica si tuvo actividad en los últimos 6 meses (1=Si/0=No)   |  
|TIENE_I_CA_D   |Tiene caja de ahorros en dolar (1=Si/0=No)   |  
|TIENE_I_CA_P   |Tiene caja de ahorros en pesos (1=Si/0=No)   |  
|TIENE_I_CBESA  |Tiene tarjeta cordobesa (1=Si/0=No)   |  
|TIENE_I_CC |Tiene cuenta corriente (1=Si/0=No)   |  
|TIENE_I_CCAD   |Tiene cuenta adelanto (1=Si/0=No)   |  
|TIENE_I_CS |Tiene cuenta social (1=Si/0=No)   |  
|TIENE_I_FCI    |Tiene Fondo Común de inversión (1=Si/0=No)   |  
|TIENE_I_PF |Tiene Plazo Fijo (1=Si/0=No)   |  
|TIENE_I_PR_HIP |Tiene préstamo hipotecario (1=Si/0=No)   |  
|TIENE_I_PR_PERS    |Tiene préstamo personal (1=Si/0=No)   |  
|TIENE_I_PREP   |Tiene tarjeta prepaga **Validar** (1=Si/0=No)   |  
|TIENE_I_SE_PATR    |Tiene seguro patrimonial (Auto/Hogar) (1=Si/0=No)   |  
|TIENE_I_SEG_VIDA   |Tiene seguro de vida (1=Si/0=No)   |  
|TIENE_I_VISA   |Tiene tarjeta VISA (1=Si/0=No)   |  
|TIENE_I_TJ_SOC |Tiene tarjeta Social (1=Si/0=No)   |  
|TIENE_E_AC_CC  |Tiene caja de ahorros y cuenta corriente? (1=Si/0=No) ** Excluir - Es solo para empresas**   |
|TIENE_E_CC |Tiene cuenta corriente? (1=Si/0=No) ** Excluir - Es solo para empresas**   |
|TIENE_E_CONV_AH    |(1=Si/0=No) ** Excluir - Es solo para empresas**   |
|TIENE_E_CONV_COMALI    |(1=Si/0=No) ** Excluir - Es solo para empresas**   |
|TIENE_E_CONV_PP    |(1=Si/0=No) ** Excluir - Es solo para empresas**   |
|TIENE_E_CONV_REC   |(1=Si/0=No) ** Excluir - Es solo para empresas**   |
|TIENE_E_CX |? (1=Si/0=No) ** Excluir - Es solo para empresas**   |
|TIENE_E_FACT   |(1=Si/0=No) ** Excluir - Es solo para empresas**   |
|TIENE_E_FCI    |Tiene Fondo Común de inversión (1=Si/0=No) ** Excluir - Es solo para empresas**   |
|TIENE_E_IB |(1=Si/0=No) ** Excluir - Es solo para empresas**   |
|TIENE_E_LIQ_COM    |(1=Si/0=No) ** Excluir - Es solo para empresas**   |
|TIENE_E_PF |Tiene Plazo Fijo? (1=Si/0=No) ** Excluir - Es solo para empresas**   |
|TIENE_E_PR_CO  |(1=Si/0=No) ** Excluir - Es solo para empresas**   |
|TIENE_E_PR_DOC |(1=Si/0=No) ** Excluir - Es solo para empresas**   |
|TIENE_E_SE |? (1=Si/0=No) ** Excluir - Es solo para empresas**   |
|TIENE_E_TC_AGRO    |Tiene tarjeta Agro (1=Si/0=No) ** Excluir - Es solo para empresas**   |
|TIENE_E_TC_CORPO   |Tiene tarjeta corporativa (1=Si/0=No) ** Excluir - Es solo para empresas**   |
|TIENE_E_TC_FLOTA   |Tiene Tarjeta Flota (1=Si/0=No) ** Excluir - Es solo para empresas**   |
|SALDO_COMPUTO_SIST_2   |Saldo Sistema Plazo Fijo   |
|SALDO_COMPUTO_SIST_CC_MB   |Saldo Cuenta Corriente Mundo BANCOR   |  
|SALDO_COMPUTO_SIST_CA_P    |Saldo Caja de ahorros Pesos   |
|SALDO_COMPUTO_SIST_CC_P    |Saldo Cuenta Corriente Pesos  |  
|SALDO_COMPUTO_SIST_FCI |Saldo en Fondos Comunes de Inversión   |
|CROSS_EMPR |Cantidad de productos en Empresas - **Excluir**   |  
|CROSS_IND  |Cantidad de productos en Individuos   |

### Proceso

In [11]:
def process_file_cs(file_input: str, file_output: str):

    uc = [
        'CUITCLIE',
        'RELACION',
        'TIPOIMPN',
        'NATUJURI',
        'SUCURSAL',
        'BCRASECT',
        'BCRASEGO',
        'GRUPO_CLIENTE',
        'GRUPOGENERAL',
        'SUBGRUPO',
        'SEGMENTO_SUELDOS',
        'FUM',
        'ES_CAPITA',
        'CL_ACTIVO_FUM',
        'TIENE_I_CA_D',
        'TIENE_I_CA_P',
        'TIENE_I_CBESA',
        'TIENE_I_CC',
        'TIENE_I_CCAD',
        'TIENE_I_CS',
        'TIENE_I_FCI',
        'TIENE_I_PF',
        'TIENE_I_PR_HIP',
        'TIENE_I_PR_PERS',
        'TIENE_I_PREP',
        'TIENE_I_SE_PATR',
        'TIENE_I_SEG_VIDA',
        'TIENE_I_VISA',
        'TIENE_I_TJ_SOC',
        'SALDO_COMPUTO_SIST_2',
        'SALDO_COMPUTO_SIST_CC_MB',
        'SALDO_COMPUTO_SIST_CA_P',
        'SALDO_COMPUTO_SIST_CC_P',
        'SALDO_COMPUTO_SIST_FCI',
        'CROSS_IND'
    ]

    tc = {
        'CUITCLIE': str,
        'RELACION': str,
        'TIPOIMPN': str,
        'NATUJURI': str,
        'SUCURSAL': str,
        'BCRASECT': str,
        'BCRASEGO': str,
        'GRUPO_CLIENTE': str,
        'GRUPOGENERAL': str,
        'SUBGRUPO': str,
        'SEGMENTO_SUELDOS': str,
        'FUM': str,
        'SALDO_COMPUTO_SIST_2': float,
        'SALDO_COMPUTO_SIST_CC_MB': float,
        'SALDO_COMPUTO_SIST_CA_P': float,
        'SALDO_COMPUTO_SIST_CC_P': float,
        'SALDO_COMPUTO_SIST_FCI': float,
    }

    # Lectura de archivo
    df_cs = pd.read_csv(file_input, 
        delimiter='|',
        encoding='latin-1',
        dtype=tc,
        parse_dates=['FUM'],
        date_parser=dateparse_yyyymmdd,
        decimal=",",
        usecols=uc)

    # Anonimizo la identificación del usuario tomando el hash de la tabla de usuarios
    df_cs = pd.merge(df_us, df_cs, how='inner', left_on='DOCUMENTO', right_on='CUITCLIE')
    df_cs.drop(['DOCUMENTO', 'CUITCLIE'], axis=1, inplace=True)

    # Reemplazo los valores SI/NO y la convierto en una variable dicotómica
    df_cs['CL_ACTIVO_FUM'].replace({'SI': 1, 'NO': 0}, inplace=True)

    # Grabo
    df_cs.to_csv(file_output, index=False)

    print(file_output, 'Filas:', len(df_cs.index)) 

In [13]:
# Proceso todos los archivos de crosseling
for periodo in PERIODOS:
    process_file_cs(
        DIR + r'\Crosselling\CLIPROD_CROSSELLING_{}.txt'.format(periodo),
        DIR + r'\Export\Crosseling_{}.csv'.format(periodo))

c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Crosseling_202202.csv Filas: 876823
c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Crosseling_202203.csv Filas: 880084
c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Crosseling_202204.csv Filas: 885787


## Depuración y generación del archivo de Gestión de Tickets

### Estructura del archivo

|Campo    |Descripción   |
|---    |---   |
|Periodo    |Perído   |
|ID |Id Gestar   |
|CASETYPEID |Id de tipo caso   |
|CASETYPE   |Descripción de tipo de caso   |
|CANORIGID  |Id de canal originante   |
|CANORIG    |Descripción de canal originante   |
|TYPE_CODSUCURSAL   |Sucursal   |
|STATEID    |Id de estado   |
|State  |Descripción de estado   |
|CREATEDDATE    |Fecha de creación del caso YYYYMMDD   |
|CREATEDMONTH   |**Excluir** Mes de creación del caso   |
|ENDDATE    |Fecha de cierre del caso   |
|SING_TIPODOC   |Tipo de documento   |
|SING_NRODOC    |Número de documento **(Ojo! A veces cargan acá el CUIT/CUIL y dejan el campo CUIT vacío)**  |
|CUIT   |CUIT   |
|CLIENTE    |**Excluir** Nombre del cliente   |
|TYPE_NROCTA    |Tipo de cuenta   |
|TYPE_NROTARJETA    |Número de cuenta   |
|NIVEL 1    |Clasificación del caso Nivel 1   |
|NIVEL 2    |Clasificación del caso Nivel 2   |
|NIVEL 3    |Clasificación del caso Nivel 3   |
|SOLVER |**Excluir** Nombre del Resolutor   |
|TYPE_MARCATARJETA  |Tipo de marca de tarjeta   |
|TPOSOLUCION_CALCULADO  |Tipo de solución   |
|TYPE_IMPORTE_TOTAL_RECLAMADO   |Importe total reclamado   |
|TYPE_IMPORTE_TOTAL_REINTEGRADO |Importe total reintegrado   |
|Motivo_Baja_Producto   |Motivo de baja del producto   |
|VIA    |**Via?**  |
|CREADO POR |**Excluir** Nombre de quien cargó el caso   |
|ID CREADOR |**Excluir** Id del creador del caso   |
|ACEPTA CONTACTO    |Acepta contacto SI/NO   |
|EMAIL  |**Excluir** Correo electrónico   |
|TYPE MOTIVO GESTION BANCON |**Motivo de gestión BANCON**   |
|DIAS_DEMORA_DETALLE    |Detalle días de demora   |
|CASETYPEPATH   |**Ruta de tipo de caso?**   |
|GRUPORESPONSABLE   |**Excluir** Grupo responsable   |
|STARTDATE  |**Fecha y hora de carga del caso?**   |
|CUSTOMERID |**Excluir** Id de cliente   |
|SING_FECHANACIMIENTO   |**SING?** **Excluir** Fecha de nacimiento   |
|SING_SEXO  |**SING?** **Excluir** Sexo   |
|SING_PROVINCIA |**SING?** **Excluir** Provincia   |
|SING_BARRIO    |**SING?** **Excluir** Barrio   |
|CUSTWORKCITY   |**Excluir** Ciudad laboural del cliente   |
|TYPE_SUCURSAL  |**Excluir** Sucursal   |
|GRUPORESPONSABLEID |**Excluir** Id de grupo resolutor   |
|TYPE_TC_TIPOTARJETA    |**TC** Tipo de tarjeta   |
|TYPE_TARJETA_NRO   |Número de tarjeta   |
|TYPE_RUBRO_COMERCIO    |Rubro del comercio   |
|type_importe_total_reclamado_dolares   |Importe total reclamado en dólares   |
|TYPE_CONVENIO  |Convenio   |
|TYPE_UBICACION |Ubicación   |
|type_tipo_plataforma_bancon    |Tipo de plataforma BANCON   |
|type_motivo_gestion_bancon_empresas    |Motivo de gestión BANCON empresas   |
|type_motivo_consulta_bancon_empresas   |Motivo consulta BANCON empresas   |
|type_motivo_consulta_bancon    |Motivo consulta BANCON   |


### Proceso

In [11]:
uc = [
    'ID',
    'CASETYPEID',
    'CASETYPE',
    'CANORIGID',
    'CANORIG',
    'TYPE_CODSUCURSAL',
    'STATEID',
    'State',
    'CREATEDDATE',
    'CREATEDMONTH',
    'ENDDATE',
    'SING_TIPODOC',
    'SING_NRODOC',
    'CUIT',
    'TYPE_NROCTA',
    'TYPE_NROTARJETA',
    'NIVEL 1',
    'NIVEL 2',
    'NIVEL 3',
    'TYPE_MARCATARJETA',
    'TPOSOLUCION_CALCULADO',
    'TYPE_IMPORTE_TOTAL_RECLAMADO',
    'TYPE_IMPORTE_TOTAL_REINTEGRADO',
    'Motivo_Baja_Producto',
    'VIA',
    'ACEPTA CONTACTO',
    'TYPE MOTIVO GESTION BANCON',
    'DIAS_DEMORA_DETALLE',
    'CASETYPEPATH',
    'STARTDATE',
    'TYPE_TC_TIPOTARJETA',
    'TYPE_TARJETA_NRO',
    'TYPE_RUBRO_COMERCIO',
    'type_importe_total_reclamado_dolares',
    'TYPE_CONVENIO',
    'TYPE_UBICACION',
    'type_tipo_plataforma_bancon',
    'type_motivo_gestion_bancon_empresas',
    'type_motivo_consulta_bancon_empresas',
    'type_motivo_consulta_bancon'
]

tc = dict(zip(uc, [str] * len(uc)))
tc.update({'TYPE_IMPORTE_TOTAL_RECLAMADO': float, 'TYPE_IMPORTE_TOTAL_REINTEGRADO': float, 'DIAS_DEMORA_DETALLE': int, 'type_importe_total_reclamado_dolares': float})

In [13]:
# Lectura de los archivos
for periodo in PERIODOS:

    file = DIR + r'\Gestar\Transacciones_Gestar_{}_NVO.txt'.format(periodo)
    df_gt = pd.read_csv(file, 
        delimiter='|',
        encoding='latin-1',
        dtype=tc,
        parse_dates=['CREATEDDATE', 'ENDDATE', 'STARTDATE'],
        usecols=uc,
        decimal=',')

    # Donde haya un CUIT con valor nulo lo plancho con el valor del DNI si su longitud de >= 11
    df_gt['CUIT'] = np.where(pd.isnull(df_gt['CUIT']) & (df_gt['SING_NRODOC'].str.len()>=11), df_gt['SING_NRODOC'], df_gt['CUIT'])
    # Borrar todas las filas sin CUIT
    df_gt = df_gt[~pd.isnull(df_gt['CUIT'])]
    # Corrijo todos los CUIT que tienen tres ceros adelante
    df_gt.loc[(df_gt['CUIT'].str.len()>11) & (df_gt['CUIT'].str[0:3] == '000'), 'CUIT'] = df_gt['CUIT'].str[3:14]
        # Corto los CUIT restantes a 11 posiciones (muchos tienen una coma detras)
    df_gt.loc[(df_gt['CUIT'].str.len()>11), 'CUIT'] = df_gt['CUIT'].str[0:11]
    
    # Le saco el guión al mes
    df_gt['CREATEDMONTH'] = df_gt['CREATEDMONTH'].str[:4] + df_gt['CREATEDMONTH'].str[5:7]
    
    # Anonimizo la identificación del usuario tomando el hash de la tabla de usuarios
    df_gt = pd.merge(df_us, df_gt, how='inner', left_on='DOCUMENTO', right_on='CUIT')
    df_gt.drop(['DOCUMENTO', 'CUIT'], axis=1, inplace=True)

    # Grabo
    file = DIR + r'\Export\Gestar_{}.csv'.format(periodo)
    df_gt.to_csv(file, index=False)
    print(file, 'Filas:', len(df_gt.index)) 

c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Gestar_202202.csv Filas: 126900
c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Gestar_202203.csv Filas: 147237
c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Gestar_202204.csv Filas: 132175


In [21]:
# Libero memoria
del df_gt

## Depuración y generación del archivo de Movimientos Monetarios y No Monetarios

### Estructura del archivo (Transacciones Monetarias y no Monetarias)

|Campo    |Descripción   |
|---    |---   |
|PERIODO    |Período YYYYMM   |
|TIPO_CANAL |Tipo de canal   |
|CANAL  |Canal (SUCURSAL/BANCON_MOBILE/BANCON_WEB)  |
|TIPO_CLIENTE   |Tipo de cliente (EMPRESA/INDIVIDUO)  |
|CUITCLIE   |CUIT/CUIL del cliente   |
|GRUPO_OPERACION    |Grupo de operación   |
|TIPO_OPERACION |Tipo de operación   |
|TIPO_OPERACION2    |Tipo de operación 2   |
|OPERACION  |Operación   |
|SUCURSAL   |Sucursal   |
|CAB    |CAB   |
|IMPORTE    |Importe   |
|ORIGEN |Origen   |
|TIPO_TRX   |Tipo de transacción (MONETARIA/NOMONETARIA)   |
|CANAL_RATIO    |**Canal Ratio?**   |
|CLASIF_TRX |Clasificación de la transacción   |

### Proceso

In [22]:
# Proceso principal de lectura, transformación y grabación del archivo de transacciones
def process_file_tx(file_input: str, file_output: str):

    uc = [
        'PERIODO',
        'TIPO_CANAL',
        'CANAL',
        'TIPO_CLIENTE',
        'CUITCLIE',
        'GRUPO_OPERACION',
        'TIPO_OPERACION',
        'TIPO_OPERACION2',
        'SUCURSAL',
        'IMPORTE',
        'ORIGEN',
        'CANAL_RATIO',
        'CLASIF_TRX'
    ]

    tc = dict(zip(uc, [str] * len(uc)))
    tc.update({'IMPORTE': float})


    # Lectura de archivo
    df_tx = pd.read_csv(file_input,
        iterator=True,
        #nrows=1_000_000,
        chunksize=1_000_000,
        delimiter='|',
        decimal=',',
        encoding='latin-1',
        dtype=tc,
        usecols=uc)

    # Itero en el archivo, lo transformo y lo voy grabando por partes
    file_mode = 'w'
    for df_tx_chunk in df_tx:

        # Primero filtro los registros con CUIT = 0 y de banca individuos
        df_tx_chunk = df_tx_chunk[(df_tx_chunk['CUITCLIE'] != '0') & (df_tx_chunk['TIPO_CLIENTE'] == 'INDIVIDUO')]

        # Anonimizo la identificación del usuario tomando el hash de la tabla de usuarios
        df_tx_chunk = pd.merge(df_us, df_tx_chunk, how='inner', left_on='DOCUMENTO', right_on='CUITCLIE')

        df_tx_chunk.drop(['DOCUMENTO', 'CUITCLIE', 'TIPO_CLIENTE'], axis=1, inplace=True)

        # Intercambio las columnas PERIODO y ID_USER
        cols_tx = list(df_tx_chunk.columns)
        col_a, col_b = cols_tx.index('ID_USER'), cols_tx.index('PERIODO')
        cols_tx[col_b], cols_tx[col_a] = cols_tx[col_a], cols_tx[col_b]
        df_tx_chunk = df_tx_chunk[cols_tx]

        # Voy grabando
        df_tx_chunk.to_csv(file_output, index=False, mode=file_mode, header=(file_mode == 'w'))
        print(file_output, 'Filas:', len(df_tx_chunk.index), file_mode, (file_mode == 'w')) 
        file_mode = 'a'

In [23]:
# Procesar los 3 archivos de movimientos que tenemos disponibles de transacciones MONETARIAS
for periodo in PERIODOS:
    process_file_tx(
        DIR + r"\Transacciones\Detalle_Trx_Monetarias_{}.txt".format(periodo),
        DIR + r"\Export\Detalle_Trx_Monetarias_{}.csv".format(periodo))

c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Detalle_Trx_Monetarias_202202.csv Filas: 865725 w True
c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Detalle_Trx_Monetarias_202202.csv Filas: 995044 a False
c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Detalle_Trx_Monetarias_202202.csv Filas: 929577 a False
c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Detalle_Trx_Monetarias_202202.csv Filas: 851290 a False
c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Detalle_Trx_Monetarias_202202.csv Filas: 666912 a False
c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Detalle_Trx_Monetarias_202202.csv Filas: 212242 a False
c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Detalle_Trx_Monetarias_202202.csv Filas: 434515 a False
c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Detalle_Trx_Monetarias_202202.csv Filas: 725269 a False
c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Detalle_Trx_Monetarias_202202.c

In [24]:
# Procesar los 3 archivos de movimientos que tenemos disponibles de transacciones NO MONETARIAS
for periodo in PERIODOS:
    process_file_tx(
        DIR + r"\Transacciones\Detalle_Trx_NoMonetarias_{}.txt".format(periodo),
        DIR + r"\Export\Detalle_Trx_NoMonetarias_{}.csv".format(periodo))

c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Detalle_Trx_NoMonetarias_202202.csv Filas: 966642 w True
c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Detalle_Trx_NoMonetarias_202202.csv Filas: 928498 a False
c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Detalle_Trx_NoMonetarias_202202.csv Filas: 1000000 a False
c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Detalle_Trx_NoMonetarias_202202.csv Filas: 1000000 a False
c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Detalle_Trx_NoMonetarias_202202.csv Filas: 999487 a False
c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Detalle_Trx_NoMonetarias_202202.csv Filas: 851731 a False
c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Detalle_Trx_NoMonetarias_202202.csv Filas: 967853 a False
c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Detalle_Trx_NoMonetarias_202202.csv Filas: 1000000 a False
c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Detalle_Trx_

## Generación del archivo de Movimientos Agregados
El motivo de la generación de este archivo es que los del apartados anterior todavía tienen un tamaño considerable para poderlos subir a Google Colab

### Proceso

In [25]:
# Proceso principal de lectura, transformación y grabación del archivo agregado de transacciones
def process_file_tx_sum(file_input: str, file_output: str):

    uc = [
        'PERIODO',
        'ID_USER',
        'TIPO_CANAL',
        'CANAL',
        'GRUPO_OPERACION',
        'TIPO_OPERACION',
        'TIPO_OPERACION2',
        'SUCURSAL',
        'ORIGEN',
        'CANAL_RATIO',
        'CLASIF_TRX',
        'IMPORTE'
    ]

    tc = dict(zip(uc, [str] * len(uc)))
    tc.update({'IMPORTE': float})

    # Lectura de archivo
    df = pd.read_csv(file_input,
        iterator=True,
        chunksize=1_000_000,
        encoding='latin-1',
        dtype=tc,
        usecols=uc)

    # Itero en el archivo, lo transformo y lo voy grabando por partes
    df_sum = pd.DataFrame()   
    for df_chunk in df:
        df_sum = pd.concat([df_sum, df_chunk.groupby([
            'PERIODO',
            'ID_USER',
            'TIPO_CANAL',
            'CANAL',
            'GRUPO_OPERACION',
            'TIPO_OPERACION',
            'TIPO_OPERACION2',
            'SUCURSAL',
            'CANAL_RATIO',
            'CLASIF_TRX',
            'ORIGEN']) \
            .aggregate(CANT_OPERACIONES=('IMPORTE', 'count'), MONTO_TOTAL=('IMPORTE', 'sum')) \
            .reset_index()])

    df_sum = df_sum.groupby([
        'PERIODO',
        'ID_USER',
        'TIPO_CANAL',
        'CANAL',
        'GRUPO_OPERACION',
        'TIPO_OPERACION',
        'TIPO_OPERACION2',
        'SUCURSAL',
        'CANAL_RATIO',
        'CLASIF_TRX',
        'ORIGEN']) \
        .aggregate('sum') \
        .reset_index()

    df_sum.to_csv(file_output, index=False)
    print(file_output, 'Filas:', len(df_sum.index)) 

In [26]:
# Procesar los 3 archivos de movimientos que tenemos disponibles de transacciones MONETARIAS
for periodo in PERIODOS:
    process_file_tx_sum(
        DIR + r'\Export\Detalle_Trx_Monetarias_{}.csv'.format(periodo),
        DIR + r'\Export\Sum_Trx_Monetarias_{}.csv'.format(periodo))

c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Sum_Trx_Monetarias_202202.csv Filas: 378149
c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Sum_Trx_Monetarias_202203.csv Filas: 403439
c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Sum_Trx_Monetarias_202204.csv Filas: 402960


In [27]:
# Procesar los 3 archivos de movimientos que tenemos disponibles de transacciones NO MONETARIAS
for periodo in PERIODOS:
    process_file_tx_sum(
        DIR + r'\Export\Detalle_Trx_NoMonetarias_{}.csv'.format(periodo),
        DIR + r'\Export\Sum_Trx_NoMonetarias_{}.csv'.format(periodo))

c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Sum_Trx_NoMonetarias_202202.csv Filas: 3504627
c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Sum_Trx_NoMonetarias_202203.csv Filas: 3714797
c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Sum_Trx_NoMonetarias_202204.csv Filas: 3862163


## Depuración y generación del Turnero

### Estructura del archivo

|Campo    |Descripción   |
|---    |---   |
|Sucursal   |Nombre Sucursal **Excluir**   |
|Proceso    |Proceso (Ej: Hablar con un asesor)   |
|Tipo_atencion  |Tipo de Atención (PRESENCIAL/VIRTUAL)   |
|Fecha_creada   |Fecha de creación (DD/MM/YYYY HH:MM:SS AM)  |
|Fecha_turno    |Fecha del turno (DD/MM/YYYY)  |
|Estado |Estado (VENCIDA/CANCELADA/FINALIZADA/PLANIFICADA)   |
|Hora_desde |Hora desde (HH:MM)  |
|Hora_hasta |Hora Hasta (HH:MM)  |
|Tipo_Doc   |Tipo de documento (DNI/LE/CUIT/LC/nan/PASAPORTE)  |
|Numero_doc |Número de documento   |
|Apellido   |Apellido **Excluir**   |
|Nombre |Nombre **Excluir**   |
|Keepchar(Telefono,'0123456789')    |Teléfono **Excluir**   |
|Email  |Correo electrénico **Excluir**   |

### Proceso

In [101]:
uc = [
    'Proceso',
    'Tipo_atencion',
    'Fecha_creada',
    'Fecha_turno',
    'Estado',
    'Hora_desde',
    'Hora_hasta',
    'Tipo_Doc',
    'Numero_doc'
]

tc = dict(zip(uc, [str] * len(uc)))

In [102]:
# Lectura de archivo
file = DIR + r'\Turnero\748_20220531_1611_Turnero_F.TXT'
df_tn = pd.read_csv(file,
    delimiter=';',
    dtype=tc,
    parse_dates=['Fecha_creada', 'Fecha_turno'],
    infer_datetime_format=True,
    usecols=uc)
df_tn.shape

(3123542, 9)

In [107]:
# Elimino los que no tienen documento
df_tn = df_tn[~df_tn['Numero_doc'].isnull()]

In [140]:
df_tn['Tipo_Doc'].unique()

array(['DNI', 'LE', 'LC'], dtype=object)

In [141]:
df_tn['Numero_doc'].str.len().unique()

array([8, 7, 6], dtype=int64)

In [110]:
# Elimina la documentos que seguro tienen basura -> Longitud <= 5
df_tn = df_tn[df_tn['Numero_doc'].str.len()>5]

In [118]:
# Elimino la documentos que tienen longitud 11 por que son casi todos CUITs de empresas
df_tn = df_tn[df_tn['Numero_doc'].str.len()!=11]

In [125]:
# Elimino todos los que tienen pasaporte porque seguro que nunca pudieron hacer el onboarding a BANCON
df_tn = df_tn[df_tn['Tipo_Doc']!='PASAPORTE']

In [130]:
# Me quedaron solo 7 registros con CUIT ni me preocupo por normalizarlos
df_tn = df_tn[df_tn['Tipo_Doc']!='CUIT']

In [139]:
# Elimino los documentos que tienen longitud 10 ya que son solo 3 registros y es un error de tipeo
df_tn = df_tn[df_tn['Numero_doc'].str.len()!=10]
# Elimino los documentos que tienen longitud 9 ya que son solo 10 registros y es un error de tipeo
df_tn = df_tn[df_tn['Numero_doc'].str.len()!=9]

In [144]:
# A lo que me quedó le pongo ceros a la izquierda
df_tn['Numero_doc'] = df_tn['Numero_doc'].apply(lambda x: x.zfill(8))

In [145]:
# Preparar la base de usuario convirtiendo CUIT/CUIL a DNI
df_us_doc = df_us.copy(deep=False)
df_us_doc['DOCUMENTO'] = df_us['DOCUMENTO'].str[2:10]

In [147]:
# Cruzo la base de usuarios con la base del turnero
df_tn = pd.merge(df_us_doc, df_tn, how='inner', left_on='DOCUMENTO', right_on='Numero_doc')
df_tn.drop(['DOCUMENTO', 'Tipo_Doc', 'Numero_doc'], axis=1, inplace=True)

In [157]:
# Grabo
file = DIR + r'\Export\Turnero.csv'
df_tn.to_csv(file, index=False)